### Uptrend Criteria

1. dataclose > ema1 and dataclose > ema2
2. (ema1 - ema2) > 1ATR (Same time frame)
3. ema1Slope > 0 ema2Slope > 0

### Downtrend Criteria

1. dataclose < ema1 and dataclose < ema2
2. (ema2 - ema1) > 1ATR (Same time frame)
3. ema1Slope > 0 ema2Slope > 0

### Conditions for Bullish Entry

In [ ]:
1. Trend Established (Bullish)
2. (datahigh>= ema1 >= datalow) or (datahigh>= ema2 >= datalow) 

In [3]:
import backtrader as bt
import backtrader.indicators as btind
from datetime import datetime


In [ ]:
class BarStrategy(bt.Strategy):
    
    params = (
        ('ema1',20),
        ('ema2',50),
        ('printlog',True),
        ('optim':False),
        ('optim_fs':(20,50))
    )
    """Logging function for this strategy"""
    def log(self, txt,dt=None,doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f"{dt.isoformat()} >> {txt}")
        
    def __init__(self):
        """Keep ref. to close in data[0] series"""
        self.dataclose = self.datas[0].close
        self.ema1 = btind.EMA(dataclose,period=self.params.ema1,plotname=f'EMA{self.params.ema1}')
        self.ema2 = btind.EMA(dataclose,period=self.params.ema2,plotname=f'EMA{self.params.ema2}')
        # To keep track of pending orders | buy price | commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.grossprofit = 0
        self.netprofit = 0
        
    def notify_order(self,order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell Order submitted/accepted to/by broker - Nothing to do
            return
        
        # Check if an order has been completed
        # Broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED! Price: {order.executed.price} | Cost: {order.executed.value} | Comm: {order.executed.comm}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log(f'SELL EXECUTED! {order.executed.price} | Cost: {order.executed.value} | Comm: {order.executed.comm}')
                
            self.bar_executed = len(self)
        
        elif order.status in [order.Cancelled,order.Margin,order.Rejected]:
            sef.log('Order Cancelled/Margin/Rejected')
            
        self.order = None
        
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        self.log(f'OPERATION PROFIT, GROSS: {trade.pnl} | NET: {trade.pnlcomm}')
        self.grossprofit+=trade.pnl
        self.netprofit+=trade.pnlcomm
        
    def next(self):
        """Adding the Logic Here"""
        # Log Current Closing Price of the Series
        self.log(f'Close: {self.dataclose[0]}')
        
        # Check if an order is pending >> Not sending another one if yes
        if self.order:
            return
        # Check if we are in the market
        if not self.position:
            # We might buy if conditions are right
            if self.dataclose[0] < self.dataclose[-1] < self.dataclose[-2]:
                # if current close < previous close < previous close (2 candle)
                self.log(f"BUY CREATE: {self.dataclose[0]}")
                self.order = self.buy()
        else:
            # Already in the market >> we might sell
            if len(self)>=(self.bar_executed + self.params.barback):
                # SELL WITH DEFAULT PARAMS
                self.log(f"SELL CREATE: {self.dataclose[0]}")
                # Keeping track of created order to avoid the second order
                self.order = self.sell()
    def stop(self):
        self.log(f"Param: {self.params.barback} >> EndingValue: {self.broker.getvalue()} | GROSS: {self.grossprofit} | NET: {self.netprofit}",doprint=True)